In [1]:
import os, json, ast
import openai
import pandas as pd
from IPython.display import display, JSON

In [38]:
# some utility methods
def search_in(foo, for_string):
    methods = dir(foo)
    return [item for item in methods if for_string.lower() in item.lower()]

def iterate_llm_response(function, argument):
    [print(function(argument)) for item in range(1,10)]

In [3]:
# since we are going to use openai's api, we need the open ai key
file_path = '../OPENAI_API_KEY.txt'
with open(file_path, 'r') as file:
    openai.api_key = file.readline()

# update the prices when you update the model you want to use, https://platform.openai.com/docs/pricing
def get_openai_model_price(model_name):
    if(model_name in 'gpt-4o-mini-2024-07-18'):
        return {'name': model_name, 'input_token_price_per_million_usd' : 0.15, 'cached_input_token_price_per_million_usd' : 0.075, 'output_token_price_per_million_usd' : 0.60}
    return None

# calculate price for a response
def get_chat_completion_stats(response):
    response_json = json.loads(response.model_dump_json())
    input_tokens = response_json['usage']['prompt_tokens']
    cached_input_tokens = response_json['usage']['prompt_tokens_details']['cached_tokens']
    output_tokens = response_json['usage']['completion_tokens']

    prices = get_openai_model_price(response_json['model'])
    input_token_price_per_million_usd = prices['input_token_price_per_million_usd']
    cached_input_token_price_per_million_usd = prices['cached_input_token_price_per_million_usd']
    output_token_price_per_million_usd = prices['output_token_price_per_million_usd']

    million = 1000000
    input_tokens_cost = (input_tokens/million)*input_token_price_per_million_usd
    cached_input_tokens_cost = (cached_input_tokens/million)*cached_input_token_price_per_million_usd
    output_token_cost = (output_tokens/million)*output_token_price_per_million_usd

    total_cost = input_tokens_cost + cached_input_tokens_cost + output_token_cost
    output = {
        'input': {
            'rate': input_token_price_per_million_usd,
            'tokens': input_tokens,
            'cost': f'{input_tokens_cost:.10f}'
        },
        'cached_input': {
            'rate': cached_input_token_price_per_million_usd,
            'tokens': cached_input_tokens,
            'cost': f'{cached_input_tokens_cost:.10f}'
        },
        'output': {
            'rate': output_token_price_per_million_usd,
            'tokens': output_tokens,
            'cost': f'{output_token_cost:.10f}'
        },
        'total': {
            'cost': f'{total_cost:.10f}'
        }
    }
    return output


import ipywidgets as widgets
from IPython.display import display, HTML
def display_stats(stats):
    data = stats
    # Create an HTML formatted string for a compact table layout
    html_content = f"""
    <style>
        .usage-table {{
            font-family: Arial, sans-serif;
            font-size: 12px;
            width: 70%;
            border-collapse: collapse;
            margin: 10px auto;
        }}
        .usage-table th, .usage-table td {{
            padding: 6px 10px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        .usage-table th {{
            background-color: #007bff;
            color: white;
            text-align: center;
        }}
        .input-row {{ background-color: #e3f2fd; }}
        .cached-row {{ background-color: #f1f8e9; }}
        .output-row {{ background-color: #ffebee; }}
        .total-row {{ background-color: #007bff; font-weight: bold; }}
    </style>
    
    <table class="usage-table">
        <tr>
            <th>Type</th>
            <th>Rate ($/1M)</th>
            <th>Tokens</th>
            <th>Cost ($)</th>
        </tr>
        <tr class="input-row">
            <td>Input 🔽</td>
            <td>{data['input']['rate']}</td>
            <td>{data['input']['tokens']}</td>
            <td>{data['input']['cost']}</td>
        </tr>
        <tr class="cached-row">
            <td>Cached Input 🔄</td>
            <td>{data['cached_input']['rate']}</td>
            <td>{data['cached_input']['tokens']}</td>
            <td>{data['cached_input']['cost']}</td>
        </tr>
        <tr class="output-row">
            <td>Output 🔼</td>
            <td>{data['output']['rate']}</td>
            <td>{data['output']['tokens']}</td>
            <td>{data['output']['cost']}</td>
        </tr>
        <tr class="total-row">
            <td>Total 💰</td>
            <td>—</td>
            <td>—</td>
            <td>{data['total']['cost']}</td>
        </tr>
    </table>
    """
    
    # Display in Jupyter Notebook
    display(HTML(html_content))

def display_response(response):
    # display(JSON(json.loads(response.model_dump_json())))
    stats = get_chat_completion_stats(response)
    display_stats(stats)

In [86]:
# let's make a simple openai api call to send in a prompt and get back a response
def get_chat_model_completions(messages):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        temperature = 0,
        max_tokens = 900
    )
    display_response(response)
    return response.choices[0].message.content

test_messages = [
    {'role': 'system', 'content': 'You are a joke teller, you give responses in less than 40 words'},
    {'role': 'user', 'content': 'tell me a joke'}
]

print(get_chat_model_completions(test_messages))

# if we see a response, it means the api-key is still working

def is_flagged_text(user_input):
    response = openai.moderations.create(input=user_input)
    flagged = response.results[0].flagged
    return flagged

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,30,0.0000045000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,18,0.0000108000
Total 💰,—,—,0.0000153000


Why did the scarecrow win an award? Because he was outstanding in his field!


In [70]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'low','Multitasking': 'high','Processing speed': 'high','Budget': '150000'}

    system_message = f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

def get_laptop_assistant():
    conversation_log = initialize_conversation()
    response = get_chat_model_completions(conversation_log)
    print(response)
    conversation_log.append({'role': 'assistant', 'content': response})
    def laptop_assistant(prompt):
        conversation_log.append({'role': 'user', 'content': prompt})
        response = get_chat_model_completions(conversation_log)
        conversation_log.append({'role': 'assistant', 'content': response})
        return response

    return laptop_assistant

    
    

In [71]:
hey_laptop_assistant = get_laptop_assistant()

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1171,0.0001756500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,30,0.0000180000
Total 💰,—,—,0.0001936500


Hello! I'm here to help you find the perfect laptop based on your needs. Please share your requirements and how you plan to use the laptop.


In [72]:
hey_laptop_assistant("Hi, I am Suyash. I am looking to buy a laptop")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1223,0.0001834500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,61,0.0000366000
Total 💰,—,—,0.0002200500


"Hi Suyash! It's great to hear that you're looking to buy a laptop. To help you find the best one for your needs, could you please tell me what you'll primarily be using the laptop for? For example, will it be for gaming, work, content creation, or something else?"

In [8]:
hey_laptop_assistant("I need a laptop for programming")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1266,0.0001899000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,74,0.0000444000
Total 💰,—,—,0.0002343000


'Great! Programming often requires a good balance of processing speed and multitasking capabilities. Could you let me know if you work with any specific programming languages or tools that might require higher performance, such as data analysis, machine learning, or game development? Additionally, do you often run multiple applications at once while coding? This will help me understand your multitasking needs better.'

In [9]:
hey_laptop_assistant("web development, software development, yes i will run multiple applications on the machine")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1362,0.0002043000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,114,0.0000684000
Total 💰,—,—,0.0003591000


'Thank you for the information, Suyash! Since you are involved in web and software development and will be running multiple applications, it sounds like you will need a laptop with high multitasking capabilities and good processing speed. \n\nNow, could you please let me know how important display quality is for you? For instance, do you prefer a high-resolution display for better visuals, or is that not a priority? Also, do you travel often with your laptop, or will you mostly be using it in a stationary location? This will help me assess the portability requirement.'

In [10]:
hey_laptop_assistant("display quality should be good, yes i would like a high resolution display. i travel quite a lot with my laptopjj")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1506,0.0002259000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,190,0.0001140000
Total 💰,—,—,0.0004359000


'Thank you for the details, Suyash! Based on your responses, I can infer the following:\n\n- **GPU intensity**: Since you are focused on programming and web development, a high GPU intensity is not a primary requirement unless you are working with graphics-heavy applications. So, I would categorize this as **low**.\n- **Display quality**: You mentioned that you prefer a high-resolution display, so this will be **high**.\n- **Portability**: Since you travel quite a lot with your laptop, portability will be **high**.\n- **Multitasking**: You indicated that you will run multiple applications, so this will be **high**.\n- **Processing speed**: Given your programming needs and multitasking, this will also be **high**.\n\nLastly, could you please share your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements.'

In [11]:
response = hey_laptop_assistant("my budget is 90000")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1709,0.0002563500
Cached Input 🔄,0.075,1664,0.0001248000
Output 🔼,0.6,151,0.0000906000
Total 💰,—,—,0.0004717500


Thank you for sharing your budget, Suyash! Based on all the information you've provided, here is the final profile for your laptop requirements:

- **GPU intensity**: low
- **Display quality**: high
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: high
- **Budget**: 90000

Here is the updated python dictionary:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 90000}
```

If you have any further questions or need more assistance, feel free to ask!


In [12]:
response = hey_laptop_assistant("i am changing my thought, i need the display quality to be really good")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1882,0.0002823000
Cached Input 🔄,0.075,1792,0.0001344000
Output 🔼,0.6,177,0.0001062000
Total 💰,—,—,0.0005229000


Thank you for the update, Suyash! Since you want the display quality to be really good, I will adjust the value for display quality to **high**. 

Let me confirm the updated values based on your latest input:

- **GPU intensity**: low
- **Display quality**: high (as per your latest input)
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: high
- **Budget**: 90000

The final updated python dictionary remains the same:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 90000}
```

If you have any more changes or questions, feel free to let me know!


In [15]:
response = hey_laptop_assistant("keep the budget to 90000")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,2262,0.0003393000
Cached Input 🔄,0.075,1408,0.0001056000
Output 🔼,0.6,153,0.0000918000
Total 💰,—,—,0.0005367000


Got it! I will keep your budget at 90,000 INR. Here’s the final profile for your laptop requirements:

- **GPU intensity**: low
- **Display quality**: high
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: high
- **Budget**: 90,000

Here is the updated python dictionary:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 90000}
```

If you have any further questions or need assistance in finding the right laptop options, feel free to ask!


In [16]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """

    response = get_chat_model_completions([{ 'role': 'system', 'content': prompt}])
    return response

hey_evaluator = intent_confirmation_layer

In [17]:
evaluator_response = hey_evaluator(response)
print(response)
print(evaluator_response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,322,0.0000483000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,2,0.0000012000
Total 💰,—,—,0.0000495000


Got it! I will keep your budget at 90,000 INR. Here’s the final profile for your laptop requirements:

- **GPU intensity**: low
- **Display quality**: high
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: high
- **Budget**: 90,000

Here is the updated python dictionary:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 90000}
```

If you have any further questions or need assistance in finding the right laptop options, feel free to ask!
Yes


In [18]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.
            Provide the response without any formatting or code block markers.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}

            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}

            Here is the input {response}

            """

    conversation = [{"role": "system", "content": prompt}]
    # print(conversation)
    response = get_chat_model_completions(conversation)
    return response

hey_python_dictionary_extractor = dictionary_present

In [19]:
specs_dict = hey_python_dictionary_extractor(response)
print(specs_dict)


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,629,0.0000943500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0001207500


{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '90000'}


In [34]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Display Type, Screen Resolution, Display Size)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM Size)",
        "Processing speed":"(CPU Type, Core, Clock Speed)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} without any code marker, for the following laptop {laptop_description}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_model_completions(messages)

    return json.loads(response)

In [21]:
laptop_description_1 = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""

In [35]:
response = product_map_layer(laptop_description_1)
print(response)
print(type(response))

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003171000


{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}
<class 'dict'>


In [39]:
iterate_llm_response(product_map_layer, laptop_description_1)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003171000


{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003171000


{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003171000


...

{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002211000


{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003171000


{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'}


In [42]:
df_laptop = pd.read_csv('laptop_data.csv')
print(df_laptop.shape)
df_laptop.head()
df_laptop['laptop_feature'] = df_laptop['Description'].apply(lambda x: product_map_layer(x))
df_laptop.to_csv('updated_laptop_data.csv', index=False, header=True)

(20, 18)


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1295,0.0001942500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002206500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1321,0.0001981500
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003205500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1271,0.0001906500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002170500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1245,0.0001867500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002131500


...

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1301,0.0001951500
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003175500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1351,0.0002026500
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003250500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1317,0.0001975500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002239500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1321,0.0001981500
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0003205500


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1339,0.0002008500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0002272500


In [59]:
import pandas as pd  # Importing the pandas library for data manipulation
import json, ast
def compare_laptops_with_user(user_req_string):
    laptop_df = pd.read_csv('updated_laptop_data.csv')

    user_requirements = ast.literal_eval(user_req_string)

    # Extracting user requirements from the input string (assuming it's a dictionary)
    # Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary

    # Extracting the budget value from user_requirements and converting it to an integer
    budget = int(user_requirements.get('Budget', '0').replace(',', '').split()[0])
    # budget
    # # Creating a copy of the DataFrame and filtering laptops based on the budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    # filtered_laptops
    # # # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {'low': 0, 'medium': 1, 'high': 2}

    # # # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_laptops['Score'] = 0

    # # # Iterating over each laptop in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = ast.literal_eval(user_product_match_str)
        # laptop_values = (dictionary_present(user_product_match_str))
        score = 0

    #     # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            # if key.lower() == 'budget':
            if key == 'Budget':
                continue  # Skipping budget comparison
            laptop_value = laptop_values.get(key, None)
            # print(key, laptop_value)
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += 1  # Incrementing score if laptop value meets or exceeds user value

        filtered_laptops.loc[index, 'Score'] = score  # Updating the 'Score' column in the DataFrame

    # Sorting laptops by score in descending order and selecting the top 3 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    top_laptops_json = top_laptops.to_json(orient='records')  # Converting the top laptops DataFrame to JSON format

    # top_laptops
    return top_laptops_json

In [54]:
specs_dict

"{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '90000'}"

In [64]:
laptop_recommendations = compare_laptops_with_user(specs_dict)
json.loads(laptop_recommendations)

[{'Brand': 'HP',
  'Model Name': 'EliteBook',
  'Core': 'i7',
  'CPU Manufacturer': 'Intel',
  'Clock Speed': '2.8 GHz',
  'RAM Size': '16GB',
  'Storage Type': 'SSD',
  'Display Type': 'LED',
  'Display Size': '14"',
  'Graphics Processor': 'Intel UHD',
  'Screen Resolution': '1920x1080',
  'OS': 'Windows 11',
  'Laptop Weight': '1.5 kg',
  'Special Features': 'Fingerprint Sensor',
  'Warranty': '3 years',
  'Average Battery Life': '8 hours',
  'Price': 90000,
  'Description': 'The HP EliteBook is a premium laptop designed for professionals who value performance and security. It features an Intel Core i7 processor clocked at 2.8 GHz, providing exceptional speed and power for demanding tasks. With 16GB of RAM and an SSD, it offers fast and efficient multitasking and storage capabilities. The laptop sports a 14" LED display with a resolution of 1920x1080, delivering sharp visuals and vibrant colors. It also comes with an Intel UHD GPU for decent graphical performance. Weighing just 1.5 

In [65]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [66]:
validated_data = recommendation_validation(laptop_recommendations)
display(validated_data,'\n')

[{'Brand': 'HP',
  'Model Name': 'EliteBook',
  'Core': 'i7',
  'CPU Manufacturer': 'Intel',
  'Clock Speed': '2.8 GHz',
  'RAM Size': '16GB',
  'Storage Type': 'SSD',
  'Display Type': 'LED',
  'Display Size': '14"',
  'Graphics Processor': 'Intel UHD',
  'Screen Resolution': '1920x1080',
  'OS': 'Windows 11',
  'Laptop Weight': '1.5 kg',
  'Special Features': 'Fingerprint Sensor',
  'Warranty': '3 years',
  'Average Battery Life': '8 hours',
  'Price': 90000,
  'Description': 'The HP EliteBook is a premium laptop designed for professionals who value performance and security. It features an Intel Core i7 processor clocked at 2.8 GHz, providing exceptional speed and power for demanding tasks. With 16GB of RAM and an SSD, it offers fast and efficient multitasking and storage capabilities. The laptop sports a 14" LED display with a resolution of 1920x1080, delivering sharp visuals and vibrant colors. It also comes with an Intel UHD GPU for decent graphical performance. Weighing just 1.5 

'\n'

In [73]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

def get_laptop_salesperson(products):
    conversation_log = initialize_conv_reco(products)
    response = get_chat_model_completions(conversation_log)
    print(response)
    conversation_log.append({'role': 'assistant', 'content': response})
    def laptop_salesperson(prompt):
        conversation_log.append({'role': 'user', 'content': prompt})
        response = get_chat_model_completions(conversation_log)
        conversation_log.append({'role': 'assistant', 'content': response})
        return response

    return laptop_salesperson

In [75]:
hey_laptop_salesperson = get_laptop_salesperson(validated_data)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,452,0.0000678000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,132,0.0000792000
Total 💰,—,—,0.0001470000


Here’s a summary of the laptop you have in your profile:

1. **HP EliteBook**: Intel Core i7 (2.8 GHz), 16GB RAM, 512GB SSD, 14" LED display (1920x1080), Intel UHD Graphics, Windows 11, Fingerprint Sensor, 1.5 kg, 8 hours battery life, 3 years warranty, ₹90,000.

Currently, it seems you only have one laptop in your profile, the HP EliteBook. If you have any specific queries about this laptop or if you're looking for recommendations or comparisons with other models, feel free to ask!


In [76]:
response = hey_laptop_salesperson("do you think its a good laptop?")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,599,0.0000898500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,273,0.0001638000
Total 💰,—,—,0.0002536500


Yes, the HP EliteBook is a solid choice, especially for professionals. Here are some reasons why it stands out:

1. **Performance**: With an Intel Core i7 processor and 16GB of RAM, it can handle demanding tasks, multitasking, and professional applications with ease.

2. **Storage**: The SSD provides fast boot times and quick access to files, enhancing overall performance.

3. **Display**: The 14" LED display with a resolution of 1920x1080 offers sharp visuals, making it suitable for presentations, video conferencing, and content consumption.

4. **Portability**: Weighing just 1.5 kg, it's lightweight and easy to carry, which is ideal for professionals on the go.

5. **Security Features**: The fingerprint sensor adds an extra layer of security, which is crucial for protecting sensitive information.

6. **Battery Life**: With an average battery life of 8 hours, it can last through a workday without needing a charge.

7. **Warranty**: A three-year warranty provides peace of mind regardin

In [97]:
def laptop_chatbot():
    
    # stage 1
    laptop_assistant = get_laptop_assistant()
    laptop_requirements = ''
    laptop_requirements_user_input = ''
    
    while(laptop_requirements_user_input != 'exit' or laptop_requirements != ''):
        laptop_requirements_user_input = input('')
        if(is_flagged_text(laptop_requirements_user_input)):
            print('Sorry, this conversation is flagged and reported, please restart your conversation')
            break

        laptop_assistant_response = laptop_assistant(laptop_requirements_user_input)
        print(laptop_assistant_response)
        if(is_flagged_text(laptop_assistant_response)):
            print('Sorry, this conversation is flagged and reported, please restart your conversation')
            break

        intent_confirmation_response = intent_confirmation_layer(laptop_assistant_response)
        print(intent_confirmation_response)
        if(is_flagged_text(intent_confirmation_response)):
            print('Sorry, this conversation is flagged and reported, please restart your conversation')
            break

        if(intent_confirmation_response == 'No'):
            continue
        elif(intent_confirmation_response == 'Yes'):
            laptop_requirements = dictionary_present(laptop_assistant_response)
            break

    # stage 2
    potential_laptops = compare_laptops_with_user(laptop_requirements)
    print("These are some potential laptops")
    print('')
    print(potential_laptops)
    
    fit_laptops = recommendation_validation(potential_laptops)
    print("These are some laptops with high scores")
    print('')
    print(fit_laptops)

    # stage 3

    laptop_salesperson = get_laptop_salesperson(fit_laptops)
    laptop_selection_user_input = ''

    while(laptop_selection_user_input != 'exit'):
        laptop_selection_user_input = input('')
        if(is_flagged_text(laptop_selection_user_input)):
            print('Sorry, this conversation is flagged and reported, please restart your conversation')
            break

        laptop_salesperson_response = laptop_salesperson(laptop_selection_user_input)
        print(laptop_salesperson_response)
        if(is_flagged_text(laptop_salesperson_response)):
            print('Sorry, this conversation is flagged and reported, please restart your conversation')
            break

laptop_chatbot()
        
        


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1171,0.0001756500
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,40,0.0000240000
Total 💰,—,—,0.0002860500


Hello! I'm here to help you find the perfect laptop based on your needs. Could you please share what you primarily plan to use the laptop for? This will help me understand your requirements better.


 Hi, I am Suyash. I need a laptop for data crunching


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1234,0.0001851000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,70,0.0000420000
Total 💰,—,—,0.0003135000


Great, Suyash! Data crunching typically requires a laptop with strong processing capabilities and good multitasking performance. To better understand your needs, could you let me know if you also work with large datasets or run complex simulations? Additionally, do you have a preference for display quality, such as needing a high-resolution screen for better data visualization?


...

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1816,0.0002724000
Cached Input 🔄,0.075,1792,0.0001344000
Output 🔼,0.6,28,0.0000168000
Total 💰,—,—,0.0004236000


You're welcome! If you have any more questions or need further assistance in the future, feel free to reach out. Happy laptop shopping!


 exit


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1852,0.0002778000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,23,0.0000138000
Total 💰,—,—,0.0002916000


Thank you for your time! If you need assistance again, don't hesitate to return. Have a great day!
